In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from mpl_toolkits import mplot3d
from collections import Counter
import math

In [2]:
root = "/home/subhash/Desktop/spam/nnfl assignment/"
data = pd.read_excel(os.path.join(root,'data4.xlsx'),header=None)

In [3]:
def normalize(data):
    for i in range(0,data.shape[1]-1):
        arr = data.iloc[:,i].to_numpy()
        data.iloc[:,i] = (arr - np.mean(arr))/np.std(arr)
    return data

In [4]:
data = normalize(data)
data = data.to_numpy()

In [5]:
def test_train_split(data,test_ratio = 0.4):
    splits = np.split(data[:, :], np.cumsum(np.unique(data[:, 7], return_counts=True)[1])[:-1])
    splits[0][:,7] = np.zeros((1,50))
    splits[1][:,7] = np.ones((1,50))
    splits[2][:,7] = np.ones((1,50))*2
    train_data = np.array((splits[0][:int(len(splits[0])*(1-test_ratio)),:],
                                 splits[1][:int(len(splits[1])*(1-test_ratio)),:],
                                 splits[2][:int(len(splits[2])*(1-test_ratio)),:]))
    test_data = np.array((splits[0][int(len(splits[0])*(1-test_ratio)):,:],
                                splits[1][int(len(splits[1])*(1-test_ratio)):,:],
                                splits[2][int(len(splits[2])*(1-test_ratio)):,:]))
    #np.random.shuffle(train_data)
    #np.random.shuffle(test_data)
    return train_data,test_data, splits

In [6]:
train_data,test_data,splits = test_train_split(data)
print(train_data.shape)
print(test_data.shape)

(3, 30, 8)
(3, 20, 8)


In [7]:
u1 = [np.mean(train_data[0][:,i]) for i in range(0,7)]
u2 = [np.mean(train_data[1][:,i]) for i in range(0,7)]
u3 = [np.mean(train_data[2][:,i]) for i in range(0,7)]

E1 = np.dot(np.transpose(train_data[0][:,:7]),train_data[0][:,:7])
E2 = np.dot(np.transpose(train_data[1][:,:7]),train_data[1][:,:7])
E3 = np.dot(np.transpose(train_data[1][:,:7]),train_data[1][:,:7])
det_E1 = np.linalg.det(E1)
det_E2 = np.linalg.det(E2)
det_E3 = np.linalg.det(E3)

In [8]:
def MAP(train_data,test_data,u1,u2,u3,E1,E2,E3,det_E1,det_E2,det_E3):

    test_data = np.concatenate((test_data[0],test_data[1],test_data[2]),axis=0)
    np.random.shuffle(test_data)
    y_pred = np.zeros((60,1))

    for i in range(len(test_data)):
        x = test_data[i][0:7]
        p_x1 = 1/((2*math.pi**2)*det_E1**-0.5)*np.exp(-0.5*(np.dot(np.dot(np.transpose(x-u1),np.linalg.inv(E1)),(x-u1))))
        p_x2 = 1/((2*math.pi**2)*det_E2**-0.5)*np.exp(-0.5*(np.dot(np.dot(np.transpose(x-u2),np.linalg.inv(E2)),(x-u2))))
        p_x3 = 1/((2*math.pi**2)*det_E3**-0.5)*np.exp(-0.5*(np.dot(np.dot(np.transpose(x-u3),np.linalg.inv(E3)),(x-u3))))

        p_x = [p_x1,p_x2,p_x3]
        y_pred[i,0] = np.argmax(p_x)

    return y_pred,test_data

In [9]:
y_pred,test_data = MAP(train_data,test_data,u1,u2,u3,E1,E2,E3,det_E1,det_E2,det_E3)

In [10]:
def test(y_pred,test_data):
    conf_matrix = np.zeros((3,3))
    ind_accuracy = np.zeros((1,3))
    overall_accuracy = 0
    for i in range(len(y_pred)):
        conf_matrix[int(y_pred[i]),int(test_data[:,7][i])] += 1
    for i in range(0,3):
        ind_accuracy[0,i] = conf_matrix[i,i]/sum(conf_matrix[i,:])
        overall_accuracy += conf_matrix[i,i]
    overall_accuracy = overall_accuracy/60
    return conf_matrix,ind_accuracy,overall_accuracy

In [11]:
conf_matrix,ind_accuracy,overall_accuracy = test(y_pred,test_data)
print("Confusion matrix :")
print(conf_matrix)
print("individual accuracy :",ind_accuracy)
print("Overall accuracy: ",np.round(overall_accuracy,decimals=2))

Confusion matrix :
[[ 5.  0.  0.]
 [15. 20.  5.]
 [ 0.  0. 15.]]
individual accuracy : [[1.  0.5 1. ]]
Overall accuracy:  0.67
